# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298572120363                   -0.85    5.2         
  2   -8.300210305607       -2.79       -1.25    1.0   97.0ms
  3   -8.300435641748       -3.65       -1.89    2.4    116ms
  4   -8.300461869262       -4.58       -2.75    3.0    162ms
  5   -8.300464095353       -5.65       -3.11    3.0    141ms
  6   -8.300464351873       -6.59       -3.28    4.2    141ms
  7   -8.300464496579       -6.84       -3.43    4.6    159ms
  8   -8.300464582156       -7.07       -3.58    5.8    236ms
  9   -8.300464624707       -7.37       -3.77    2.2    173ms
 10   -8.300464638605       -7.86       -3.95    1.9    133ms
 11   -8.300464643643       -8.30       -4.28    1.9    140ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67494264370                   -0.70    6.5         
  2   -16.67874040217       -2.42       -1.14    1.2    264ms
  3   -16.67921714703       -3.32       -1.86    2.4    306ms
  4   -16.67927916964       -4.21       -2.76    2.0    278ms
  5   -16.67928592405       -5.17       -3.18    5.9    416ms
  6   -16.67928619716       -6.56       -3.48    3.6    345ms
  7   -16.67928621831       -7.67       -3.94    1.8    255ms
  8   -16.67928622142       -8.51       -4.58    3.1    308ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32586879434                   -0.56    7.0         
  2   -33.33413243658       -2.08       -1.00    1.2    974ms
  3   -33.33589610663       -2.75       -1.73    6.5    1.40s
  4   -33.33613757704       -3.62       -2.63    6.1    1.26s
  5   -33.33687622408       -3.13       -2.46    5.9    1.55s
  6   -33.33691331639       -4.43       -2.64    2.5    969ms
  7   -33.33694332408       -4.52       -3.57    2.6    1.14s
  8   -33.33694367936       -6.45       -3.81    4.9    1.44s
  9   -33.33694378193       -6.99       -4.66    1.9    988ms


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298129046449                   -0.85    5.1         
  2   -8.300201221426       -2.68       -1.59    1.0   94.0ms
  3   -8.300353824896       -3.82       -2.25    2.4    103ms
  4   -8.300324646063   +   -4.53       -2.18    3.2    132ms
  5   -8.300463622722       -3.86       -3.37    1.0   77.7ms
  6   -8.300464467636       -6.07       -3.64   10.2    234ms
  7   -8.300464619487       -6.82       -3.99    1.4   90.5ms
  8   -8.300464626502       -8.15       -4.09    1.8   98.8ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32500446428                   -0.56    6.9         
  2   -33.27203175155   +   -1.28       -1.25    1.1    790ms
  3   +16.49757327588   +    1.70       -0.22    8.9    2.07s
  4   -33.30494164769        1.70       -1.57    6.5    1.89s
  5   -33.24089382020   +   -1.19       -1.33    4.2    1.29s
  6   -32.33370208351   +   -0.04       -1.06    3.4    1.23s
  7   -33.29099228037       -0.02       -1.71    4.6    1.23s
  8   -33.33504160090       -1.36       -2.18    2.8    985ms
  9   -33.33486016448   +   -3.74       -2.09    2.6    1.23s
 10   -33.33676755399       -2.72       -2.74    2.0    966ms
 11   -33.33692867419       -3.79       -2.97    2.4    1.03s
 12   -33.33693769229       -5.04      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.